## IN4640 Assignment 1
### Intensity Transformations and Neighborhood Filtering
### Task 04

In [3]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# Load the image
img_path = 'assets/looking_out.jpg'

if not os.path.exists(img_path):
    print(f"Error: File not found at {img_path}")
else:
    # Convert to grayscale
    img_bgr = cv2.imread(img_path)
    img_gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    
    # (a) Use Otsu thresholding to obtain the binary mask for the foreground 
    # comprising the woman and the room.
    # The woman and room are likely darker than the bright window.
    # Otsu's thresholding returns the threshold value and the thresholded image.
    # cv2.THRESH_BINARY yields 255 for pixels > threshold (bright window).
    # We invert it (cv2.THRESH_BINARY_INV) to get the dark regions (foreground) as 255.
    ret, mask = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    
    print(f"Otsu's Threshold Value: {ret}")

    # (b) Carry out histogram equalization only for the foreground region.
    
    def equalize_masked_region(image, region_mask):
        # Extract the values of the pixels in the masked region
        # region_mask is 255 for relevant pixels
        valid_pixels = image[region_mask == 255]
        
        # Calculate histogram for these pixels
        hist, bins = np.histogram(valid_pixels.flatten(), 256, [0, 256])
        
        # Calculate CDF
        cdf = hist.cumsum()
        cdf_normalized = cdf * hist.max() / cdf.max()
        
        # Mask CDF to create mapping
        cdf_m = np.ma.masked_equal(cdf, 0)
        cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
        cdf_final = np.ma.filled(cdf_m, 0).astype('uint8')
        
        # Create output image starting as copy of original
        equalized_img = image.copy()
        
        # Map values only in the masked region
        # We need to apply the mapping function `cdf_final` to the pixel values
        # The pixel values in the masked region are `image[region_mask == 255]`
        # The mapped values are `cdf_final[image[region_mask == 255]]`
        equalized_img[region_mask == 255] = cdf_final[image[region_mask == 255]]
        
        return equalized_img

    result_img = equalize_masked_region(img_gray, mask)

    # Visualization
    plt.figure(figsize=(15, 10))

    plt.subplot(2, 2, 1)
    plt.title("Original Grayscale")
    plt.imshow(img_gray, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 2, 2)
    plt.title(f"Foreground Mask (Otsu Thresh={ret})")
    plt.imshow(mask, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 2, 3)
    plt.title("Foreground Equalized")
    plt.imshow(result_img, cmap='gray')
    plt.axis('off')

    plt.subplot(2, 2, 4)
    plt.title("Histograms")
    plt.hist(img_gray[mask==255].flatten(), 256, [0, 256], color='gray', alpha=0.5, label='Original Foreground')
    plt.hist(result_img[mask==255].flatten(), 256, [0, 256], color='blue', alpha=0.5, label='Equalized Foreground')
    plt.legend()
    
    plt.tight_layout()
    plt.show()